# Deep Learning model to detect suspiciously named files

This notebook uses a pre-trained Deep Learning model to predict whether a processname is suspicious or not.

Malwares and malicious programs such as ransomware often use tactics, techniques, and procedures (TTPs) such as copying malicious files to the local machine to propagate themselves across the network. A key indicator of compromise is that after a successful execution of the malware, it copies itself as an executable file with a randomly generated filename and places this file in one of the directories. Such techniques are seen in several malwares such as TrickBot. 

We develop machine learning model that uses a Recurrent Neural Network (RNN) to distinguish between malicious and benign processnames. The model is trained independently and is then made available for download. We use a character level RNN to classify malicious vs. benign processnames. Higher the `is_malicious_prob`, the more likely is the processname to be suspicious (between `[0,1]`). The threshold for flagging a processname as suspicious is set at `0.5`.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# This definition exposes all python module imports that should be available in all subsequent commands

import pandas as pd
import numpy as np
import random
import string
import torch
import torch.nn as nn

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/detect_suspicious_processnames_using_pretrained_model_in_dsdl/detect_suspicious_processnames_using_pretrained_model_in_dsdl.pt"

# model parameters
all_letters = string.ascii_lowercase
n_letters = len(all_letters)
n_hidden = 32
n_categories = 2
learning_rate = 5e-4

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.23.3
pandas version: 1.5.0


## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DSDL Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DSDL app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter.

### Option 1 - pull data from Splunk

In [5]:
from dsdlsupport import SplunkSearch as SplunkSearch

In [6]:
search = SplunkSearch.SplunkSearch()

In [7]:
df = search.as_df()
df

,_time,feature_0,feature_1,feature_2,i,s
0,2022-10-12T15:58:22.000+02:00,0.2428163280710578,0.2446760144084692,0.878678847104311,1,1
1,2022-10-12T15:58:22.000+02:00,0.458672939799726,0.4403486642986536,0.5951908086426556,2,2
2,2022-10-12T15:58:22.000+02:00,0.4268499561585486,0.70259142620489,0.6676120953634381,3,0
3,2022-10-12T15:58:22.000+02:00,0.9911085437051952,0.7537638866342604,0.8423880040645599,4,1
4,2022-10-12T15:58:22.000+02:00,0.2864162167534232,0.4240174447186291,0.7597004366107285,5,2
5,2022-10-12T15:58:22.000+02:00,0.9403447071090341,0.10794003773480654,0.22778158355504274,6,0
6,2022-10-12T15:58:22.000+02:00,0.6541405455209315,0.7657363642938435,0.07344387145712972,7,1
7,2022-10-12T15:58:22.000+02:00,0.8196326880715787,0.34731235541403294,0.8556739939376712,8,2
8,2022-10-12T15:58:22.000+02:00,0.3521619248203933,0.04771727975457907,0.07742329593747854,9,0
9,2022-10-12T15:58:22.000+02:00,0.2476972616277635,0.5393188064917922,0.7553935945034027,10,1


### Option 2 - push data from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. You utilize the `mode=stage` flag in the in the `| fit` command to do this. The search results are accessible then as csv file with the same model name that is defined in the `into app:<modelname>` part of the fit statement. Additionally, meta data is retrieved and accessible as json file. In the same way you can further work with the meta data object as it is exposed in the fit and apply function definitions below in stage 3 and 4.

| makeresults count=10<br>
| streamstats c as i <br>
| eval s = i%3 <br>
| eval feature_{s}=0 <br>
| foreach feature_* [eval \<\<FIELD\>\>=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone_template _time feature_* i into app:barebone_template<br>

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [7]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [10]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("barebone_template")

In [11]:
df

,_time,i,feature_1,feature_2,feature_0
0,1665565121,1,0.749423,0.861980,0.063747
1,1665565121,2,0.328421,0.494055,0.578063
2,1665565121,3,0.456588,0.135485,0.935327
3,1665565121,4,0.034384,0.457241,0.043726
4,1665565121,5,0.319659,0.867864,0.595515
...,...,...,...,...,...
995,1665565121,996,0.171754,0.162497,0.923288
996,1665565121,997,0.500971,0.899659,0.607057
997,1665565121,998,0.209019,0.751987,0.469225
998,1665565121,999,0.163475,0.825084,0.158101


In [8]:
param

{'options': {'params': {'mode': 'stage', 'algo': 'barebone_template'},
  'args': ['_time', 'feature_*', 'i'],
  'feature_variables': ['_time', 'feature_*', 'i'],
  'model_name': 'barebone_template',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '600',
   'max_inputs': '100000',
   'max_memory_usage_mb': '4000',
   'max_model_size_mb': '30',
   'max_score_time': '600',
   'use_sampling': 'true'},
  'kfold_cv': None},
 'feature_variables': ['_time', 'feature_1', 'feature_2', 'feature_0', 'i']}

## Stage 2 - create and initialize a model

In [12]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently

class ProcessnameClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ProcessnameClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

    
def init(df, param):
    model = ProcessnameClassifier(n_letters, n_hidden, n_categories)
    model.load_state_dict(torch.load(MODEL_DIRECTORY, map_location=torch.device('cpu')))
    model.eval()
    
    return model
    

In [13]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'hyperparameter': 42.0}


## Stage 3 - fit the model

In [15]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [16]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [17]:
# apply your model
# returns the calculated results

# Find letter index from all_letters
def letterToIndex(letter):
    return all_letters.find(letter)


# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


def evaluate(model, line_tensor):
    hidden = model.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = model(line_tensor[i], hidden)

    return output


def preprocess_txt(input_line):
    input_line = input_line.split('/')[-1]
    input_line = input_line.lower()
    input_line = input_line.replace('.exe','')
    input_line = ''.join(filter(str.islower, input_line))
    
    return input_line


def apply(model, df, param):
    is_malicious_prob_lst = []
    for idx, row in df.iterrows():
        input_line = row['text']
        input_line = preprocess_txt(input_line)
        
        if len(input_line) == 0:
            is_mal_prob = 1
        else:
            with torch.no_grad():
                output = evaluate(model, lineToTensor(input_line))

            output = torch.exp(output)
            is_mal_prob = round(output[0][1].item(), 4)

        is_malicious_prob_lst.append(is_mal_prob)
    
    output = pd.DataFrame()
    output['is_malicious_prob'] = is_malicious_prob_lst

    return output

In [18]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

     index
0        0
1        1
2        2
3        3
4        4
..     ...
995    995
996    996
997    997
998    998
999    999

[1000 rows x 1 columns]


## Stage 5 - save the model

In [15]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [16]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = ProcessnameClassifier(n_letters, n_hidden, n_categories)
    model.load_state_dict(torch.load(MODEL_DIRECTORY, map_location=torch.device('cpu')))
    model.eval()
    return model 

## Stage 7 - provide a summary of the model

In [17]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code